In [ ]:
import dill
import torch
import matplotlib.pyplot as plt
from src.PointwiseNet import PointwiseFNN
from src.GridbasedNet import GridbasedFNN
from torch.utils.data import DataLoader

def Savepickle(obj, doc_path):
  with open(doc_path, 'wb') as file:
         dill.dump(obj, file)     

def Readpickle(doc_path):
    with open(doc_path, 'rb') as file:
        return dill.load(file)

# European Call 
## Point-wise Learning

### Data Loader

In [ ]:
# Read data
data_saved = Readpickle('./data/European/point_wise_training_data.pkl')
paras = data_saved['paras']
para_index = data_saved['para_index']
dataset = data_saved['Xnormalized_data']
train, validate, test = data_saved['train'], data_saved['validate'], data_saved['test']
loader = { 'train' : DataLoader(train, batch_size=32, shuffle=True),
           'validate': DataLoader(validate, batch_size=32, shuffle=True),
           'test'  : DataLoader(test,  batch_size=32, shuffle=True) }

### Network Training

In [ ]:
# Train the model
# from src.PointwiseNet import PointwiseFNN
point_wise_IV_approximator = PointwiseFNN(loader)
point_wise_IV_approximator.Train(num_epochs=200, learning_rate=0.01, lr_step_size=10, min_lr =5e-4, abs_tolerance=0.001)
point_wise_IV_approximator.Test(loader['test'])

In [ ]:
# Reletive error on test
point_wise_IV_approximator.eval()
with torch.no_grad():
    y_predict = point_wise_IV_approximator(test[:][0])
    y_label = test[:][1]
    relative_error = torch.abs((y_label - y_predict)/y_label).flatten().numpy() 
    absolute_error = torch.abs((y_label - y_predict)).flatten().numpy()

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.hist(relative_error, bins=30, range=(0, 0.1), alpha=0.5, color='blue', edgecolor='black')
plt.xlabel('Relative_Error')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
plt.hist(absolute_error, bins=30, range=(0, 0.1), alpha=0.5, color='blue', edgecolor='black')
plt.xlabel('Absolute_Error')
plt.ylabel('Frequency')

plt.show()

In [ ]:
# Save the model
result_saved = {
    'state_dict' : point_wise_IV_approximator.state_dict(),
    'train_loss' : point_wise_IV_approximator.train_loss,
    'validate_loss' : point_wise_IV_approximator.validate_loss
}

# Savepickle(result_saved, './data/European/point_wise_training_result.pkl')

In [ ]:
# Read the model
result_saved = Readpickle('./data/European/point_wise_training_result.pkl')
point_wise_IV_approximator = PointwiseFNN(loader)
point_wise_IV_approximator.load_state_dict(result_saved['state_dict'])
point_wise_IV_approximator.train_loss = result_saved['train_loss']
point_wise_IV_approximator.validate_loss = result_saved['validate_loss']
point_wise_IV_approximator.Test(loader['test'])

## Grid-based Learning

### Data loader

In [ ]:
# Read data
data_saved = Readpickle('./data/European/grid_based_training_data.pkl')
paras = data_saved['paras']
para_index = data_saved['para_index']
dataset = data_saved['Xnormalized_data']
train, validate, test = data_saved['train'], data_saved['validate'], data_saved['test']
loader = { 'train' : DataLoader(train, batch_size=32, shuffle=True),
           'validate': DataLoader(validate, batch_size=32, shuffle=True),
           'test'  : DataLoader(test,  batch_size=32, shuffle=True) }
recover_y_dim = data_saved['recover_y_dim'] # 88

### Network training

In [ ]:
# Train the model
grid_based_IV_approximator = GridbasedFNN(loader)
grid_based_IV_approximator.Train(num_epochs=15000, learning_rate=0.001, lr_step_size=100, min_lr =5e-4, abs_tolerance=0.001)
grid_based_IV_approximator.Test(loader['test'])

In [ ]:
grid_based_IV_approximator.eval()
with torch.no_grad():
    y_predict = grid_based_IV_approximator(test[:][0])
    y_label = test[:][1]
    relative_error = torch.abs((y_label - y_predict)/y_label) # .numpy().reshape(recover_y_dim)
    relative_error_max = relative_error.max(dim=0)[0].numpy().reshape(recover_y_dim)
    relative_error_mean = relative_error.mean(dim=0).numpy().reshape(recover_y_dim)
    relative_error_median = relative_error.median(dim=0)[0].numpy().reshape(recover_y_dim)
    
    abs_error = torch.abs((y_label - y_predict)) # .numpy().reshape(recover_y_dim)
    abs_error_max = abs_error.max(dim=0)[0].numpy().reshape(recover_y_dim)
    abs_error_mean = abs_error.mean(dim=0).numpy().reshape(recover_y_dim)
    abs_error_median = abs_error.median(dim=0)[0].numpy().reshape(recover_y_dim)

In [ ]:
# Save the model
result_saved = {
    'state_dict' : grid_based_IV_approximator.state_dict(),
    'train_loss' : grid_based_IV_approximator.train_loss,
    'validate_loss' : grid_based_IV_approximator.validate_loss
}

# Savepickle(result_saved, './data/European/grid_based_training_result.pkl')

In [ ]:
# Read the model
result_saved = Readpickle('./data/European/grid_based_training_result.pkl')
grid_based_IV_approximator = GridbasedFNN(loader)
grid_based_IV_approximator.load_state_dict(result_saved['state_dict'])
grid_based_IV_approximator.train_loss = result_saved['train_loss']
grid_based_IV_approximator.validate_loss = result_saved['validate_loss']
grid_based_IV_approximator.Test(loader['test'])